# 🚀​ Funciones requeridas para la API

En este notebook se desarrollan y prueban las funciones que son solicitadas para la API. Luego, al momento de deployarlas es posible que sufran algunos cambios en función de los requerimientos propios de FastAPI o Render.

#### 📥Importaciones 

In [13]:
import pandas as pd 
import json

#### 📦 Extracción de los dataset necesarios

Se cargan los conjuntos de datos necesarios para llevar a cabo las consultas solicitadas. Se decide subir los datos de esta manera, y no los dataset completos, debido a la limitada capacidad de almacenamiento que permite Render. Por otra parte, considerando que el objetivo es entregar un PVM, a modo de prueba de concepto, se consideró adecuada esta simplificación de los datasets.

Los  dataset que se utilizan son:

* **df_games_unido**: contiene datos relacionados con los juegos en sí, como los títulos, el desarrollador, los precios, las características técnicas, las etiquetas, entre otros datos.
* **df_items_unido**: contiene la información sobre los juegos que juegan todos los usuarios, así como el tiempo acumulado que cada usuario jugó a un determinado juego.
* **df_reviews_unido**: contiene la información relacionada a los usuarios que realizan reviews de los juegos en formato de análisis de sentimientos. Entre esta información, se encuentran las recomendaciones o no del juego por parte de usuario, la fecha del review así como datos del usuario como su id, su url del perfil.

In [2]:
df_juegos = pd.read_csv('data/df_games_unido.csv')
df_items = pd.read_csv('data/df_items_unido.csv')
df_reviews = pd.read_csv('data/df_reviews_unido.csv')

A continuación, se desarrollan cada una de las funciones solicitadas.

* PlayTimeGenre

Se crea la función 'PlayTimeGenre' que debe devolver el año con mas horas jugadas para dicho género.

In [3]:
def PlayTimeGenre(genero, df_juegos, df_items):
    # Se filtra el DataFrame de juegos por el género dado
    juegos_genero = df_juegos[df_juegos['genres'].str.contains(genero)]

    # Se combinan los DataFrames de juegos y items en función del id
    df_combinado = pd.merge(df_items, juegos_genero, left_on='item_id', right_on='id')

    # Se encuentra el año con más horas jugadas para el género dado
    year_most_played = df_combinado.groupby('release_anio')['playtime_forever'].sum().idxmax()

    return {"Año de lanzamiento con más horas jugadas para " + genero: year_most_played}

In [4]:
PlayTimeGenre('Action', df_juegos, df_items)

{'Año de lanzamiento con más horas jugadas para Action': '2012'}

* UserForGenre

Se crea la función 'UserForGenre' que debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

In [5]:
def UserForGenre(genero, df_juegos, df_items):
    # Se filtra el DataFrame de juegos por el género dado
    juegos_genero = df_juegos[df_juegos['genres'].str.contains(genero)]

    # Se combinan los DataFrames de juegos y items en función del id
    df_combinado = pd.merge(df_items, juegos_genero, left_on='item_id', right_on='id')

    # Se encuentra el usuario con más horas jugadas para el género dado
    user_most_played = df_combinado.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Se acumulan las horas jugadas por año para el usuario más activo en ese género
    horas_por_anio = df_combinado[df_combinado['user_id'] == user_most_played].groupby('release_anio')['playtime_forever'].sum().reset_index()
    lista_horas_anio = [{"Año": year, "Horas": hours} for year, hours in zip(horas_por_anio['release_anio'], horas_por_anio['playtime_forever'])]

    return {"Usuario con más horas jugadas para " + genero: user_most_played, "Horas jugadas": lista_horas_anio}

In [6]:
UserForGenre('Casual', df_juegos, df_items)

{'Usuario con más horas jugadas para Casual': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': '1999', 'Horas': 0},
  {'Año': '2002', 'Horas': 0},
  {'Año': '2004', 'Horas': 6985},
  {'Año': '2007', 'Horas': 0},
  {'Año': '2008', 'Horas': 1241},
  {'Año': '2009', 'Horas': 2870},
  {'Año': '2010', 'Horas': 12066},
  {'Año': '2011', 'Horas': 81201},
  {'Año': '2012', 'Horas': 29722},
  {'Año': '2013', 'Horas': 107965},
  {'Año': '2014', 'Horas': 122364},
  {'Año': '2015', 'Horas': 362201},
  {'Año': '2016', 'Horas': 498260},
  {'Año': '2017', 'Horas': 58},
  {'Año': 'Dato no disponible', 'Horas': 0}]}

* UsersRecommend

Se crea la función 'UsersRecommend' que devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.

In [7]:
def UsersRecommend(año, df_reviews):
    # Se filtran las reviews para el año dado y que sean positivas o neutrales
    reviews_año = df_reviews[df_reviews['release_anio'] == año]
    reviews_positivas_neutrales = reviews_año[reviews_año['sentiment_analysis'] >= 1]

    # Se cuentan las recomendaciones por juego
    recomendaciones_por_juego = reviews_positivas_neutrales[reviews_positivas_neutrales['reviews_recommend'] == True]['title'].value_counts().reset_index()
    recomendaciones_por_juego.columns = ['Juego', 'reviews_recommend']

    # Se obtiene el top 3 de juegos más recomendados
    top_3_juegos = recomendaciones_por_juego.head(3).to_dict('records')

    return [{"Puesto " + str(idx + 1): juego} for idx, juego in enumerate(top_3_juegos)]

In [8]:
UsersRecommend('2017', df_reviews)

[{'Puesto 1': {'Juego': 'Unturned', 'reviews_recommend': 874}},
 {'Puesto 2': {'Juego': 'Robocraft', 'reviews_recommend': 241}},
 {'Puesto 3': {'Juego': 'ARK: Survival Evolved', 'reviews_recommend': 215}}]

* UsersWorstDeveloper

Se crea la función 'UsersWorstDeveloper' que devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado.

In [9]:
def UsersWorstDeveloper(año, df_reviews):
    # Se filtran las reviews para el año dado, que sean negativas y no recomendadas
    reviews_año = df_reviews[df_reviews['release_anio'] == año]
    reviews_negativas_no_recomendadas = reviews_año[(reviews_año['sentiment_analysis'] == 0) & (reviews_año['reviews_recommend'] == False)]

    # Se cuentan las recomendaciones por desarrollador
    recomendaciones_por_desarrollador = reviews_negativas_no_recomendadas['developer'].value_counts().reset_index()
    recomendaciones_por_desarrollador.columns = ['Desarrollador', 'Count']

    # Se obtiene el top 3 de desarrolladoras con menos recomendaciones negativas
    top_3_desarrolladoras = recomendaciones_por_desarrollador.tail(3).to_dict('records')

    return [{"Puesto " + str(idx + 1): desarrollador} for idx, desarrollador in enumerate(top_3_desarrolladoras)]

In [10]:
UsersWorstDeveloper('2017', df_reviews)

[{'Puesto 1': {'Desarrollador': 'oddonegames', 'Count': 1}},
 {'Puesto 2': {'Desarrollador': 'Pixel Dash Studios, EQgames', 'Count': 1}},
 {'Puesto 3': {'Desarrollador': 'Alientrap', 'Count': 1}}]

* SentimentAnalysis

Se crea la función 'sentiment_analysis' que según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

In [11]:
def sentiment_analysis(empresa_desarrolladora, df_reviews):
    # Se transforman los valores de 'sentiment_analysis' a las categorías de sentimientos
    mapeo_sentimientos = {0: 'Negativo', 1: 'Neutral', 2: 'Positivo'}
    df_reviews['sentiment_analysis'] = df_reviews['sentiment_analysis'].map(mapeo_sentimientos)

    # Se filtran las reviews por la empresa desarrolladora dada
    reviews_empresa = df_reviews[df_reviews['developer'] == empresa_desarrolladora]

    # Se cuenta el número de reviews en cada categoría de análisis de sentimientos
    conteo_sentimientos = reviews_empresa['sentiment_analysis'].value_counts().to_dict()

    # Se crea el diccionario de salida
    salida = {empresa_desarrolladora: conteo_sentimientos}

    return salida

In [12]:
sentiment_analysis('Valve', df_reviews)

{'Valve': {'Neutral': 5503, 'Positivo': 3222, 'Negativo': 711}}